## 6.2 Auto Enocder模型

自编码器是很经典的模型，分为encoder和decoder两个部分，特征输入encoder后输出编码，这个编码又做为decoder端的输入，经过decoder后做预测。

下面时一个非常简单的自编码器的构建与训练。

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class AutoEncoderDecoder(nn.Module):
    def __init__(self, dim_in, dim_out=128, dropout=0.2):
        """模型结构"""
        super(AutoEncoderDecoder, self).__init__()

        self.encoder = nn.Sequential(
            nn.Linear(dim_in, dim_out),
            nn.LeakyReLU()
        )

        self.decoder = nn.Sequential(
            nn.Linear(dim_out, dim_in)
        )

        self.dropout = dropout

    def forward(self, x):
        e = self.encoder(x)
        e = F.dropout(e, self.dropout)   # 可以增加dropout, 也可以去掉，将一定比例的数据设置为0
        d = self.decoder(e)

        return e, d


class TrainAE():
    def __init__(self, epoch=5):
        """训练，将X输入encoder->decoder，最小化输出与输入的损失，及让自己生成自己"""
        self.epoch = epoch

    def train(self, X):
        # 创建模型
        N, dim_in = X.shape()
        model = AutoEncoderDecoder(dim_in)

        # 优化器
        optim = torch.optim.Adam(model.parameters(), lr=0.0001)

        # 损失
        loss_f = torch.nn.MSELoss()

        for e in range(self.epoch):
            optim.zero_grad()

            embedding, Y_pred = model(X)
            loss = loss_f(Y_pred, X)

            loss.backward()

            optim.step()

        embedding, Y_pred = model(X)
        return embedding.data.numpy()